# Simulation vs Real Robot - Same Code!

This notebook demonstrates how the same control code works for both simulation and real hardware.

## Setup

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import numpy as np
import time
import matplotlib.pyplot as plt

## Choose: Simulation or Real Robot

**Change `USE_REAL_ROBOT` to switch between them!**

In [2]:
# Toggle this to switch between simulation and real hardware
USE_REAL_ROBOT = False  # Set to True for real robot

if USE_REAL_ROBOT:
    # Real hardware - connect to robot
    from applied_planning.hardware import XArmLite6Adapter
    robot = XArmLite6Adapter(robot_ip='192.168.1.161')
    print("✓ Using REAL ROBOT at 192.168.1.161")
else:
    # Simulation - load MuJoCo model
    from applied_planning.sim.adapters import MujocoLite6Adapter
    model_path = '../src/applied_planning/sim/assets/ufactory_lite6/lite6.xml'
    robot = MujocoLite6Adapter(model_path, viewer=False, render_mode='offscreen')
    print("✓ Using SIMULATION")

IK solver using site: attachment_site
Offscreen renderer initialized (notebook-friendly mode)
✓ Using SIMULATION


## Test 1: Your Original Code

**This is your exact code - works for both!**

In [3]:
# Your original code
robot.reset()
robot.step({'qvel': [0.1, 0, 0, 0, 0, 0]})

print("After first step:")
print(f"  Joint angles: {np.rad2deg(robot.get_state()['qpos'])}")
print(f"  EE position: {robot.get_ee_position()}")

After first step:
  Joint angles: [ 8.65224567e-04  9.46092562e-04 -9.08016772e-04  1.54538961e-05
 -1.07860161e-05  4.18970321e-07]
  EE position: [ 8.69926986e-02 -5.52432104e-07  1.53586816e-01]


In [4]:
robot.reset()
robot.step({'qvel': [0, 0.1, 0.1, 0, 0, 0]})

print("After second step:")
print(f"  Joint angles: {np.rad2deg(robot.get_state()['qpos'])}")
print(f"  EE position: {robot.get_ee_position()}")

After second step:
  Joint angles: [ 4.26374693e-05  1.70757602e-03 -1.42942736e-04 -1.09717791e-06
 -6.21171936e-06  1.03447399e-07]
  EE position: [ 8.69953725e-02 -1.80136298e-06  1.53586821e-01]


## Test 2: Longer Motion

Move for 2 seconds to see more motion

In [ ]:
robot.reset()
print("Moving joint 1 for 2 seconds...\n")

# Record trajectory
trajectory = []

for i in range(200):  # 200 steps × 0.01s = 2 seconds
    robot.step({'qvel': [0.2, 0, 0, 0, 0, 0]})  # 0.2 rad/s
    
    state = robot.get_state()
    trajectory.append({
        'time': i * 0.01,
        'joint1': np.rad2deg(state['qpos'][0]),
        'ee_pos': robot.get_ee_position().copy()
    })
    
    if i % 50 == 0:
        print(f"t={i*0.01:.1f}s: Joint 1 = {trajectory[-1]['joint1']:.1f}°")

print(f"\n✓ Final angle: {trajectory[-1]['joint1']:.1f}°")
print(f"✓ Total rotation: {trajectory[-1]['joint1'] - trajectory[0]['joint1']:.1f}°")

## Visualize Trajectory

In [ ]:
# Plot joint angle over time
times = [d['time'] for d in trajectory]
angles = [d['joint1'] for d in trajectory]

plt.figure(figsize=(10, 6))
plt.plot(times, angles, linewidth=2)
plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Joint 1 Angle (degrees)', fontsize=12)
plt.title('Joint 1 Motion Over Time', fontsize=14)
plt.grid(True, alpha=0.3)
plt.show()

print(f"Motion type: {'REAL ROBOT' if USE_REAL_ROBOT else 'SIMULATION'}")

## Test 3: All Joints Moving

Move all 6 joints at different speeds

In [ ]:
robot.reset()
print("Moving all joints for 3 seconds...\n")

# Different velocity for each joint
velocities = np.array([0.1, 0.15, 0.2, 0.1, 0.15, 0.1])  # rad/s

all_joint_traj = []

for i in range(300):  # 3 seconds
    robot.step({'qvel': velocities.tolist()})
    
    if i % 100 == 0:
        state = robot.get_state()
        angles = np.rad2deg(state['qpos'])
        print(f"t={i*0.01:.1f}s: {angles}")
        all_joint_traj.append(angles)

print("\n✓ Motion complete!")

## Visualize Current State

In [ ]:
if not USE_REAL_ROBOT:
    # Only works in simulation
    img = robot.render_notebook(width=800, height=600)
    plt.figure(figsize=(12, 9))
    plt.imshow(img)
    plt.axis('off')
    plt.title(f'Robot State - EE: {robot.get_ee_position()}')
    plt.show()
else:
    print("Visualization only available in simulation")
    print(f"Current EE position: {robot.get_ee_position()}")
    state = robot.get_state()
    print(f"Current joint angles: {np.rad2deg(state['qpos'])}°")

## Cleanup

In [ ]:
robot.close()
print("✓ Disconnected")

## Summary

**The key insight:** Only the initialization changes between simulation and real robot!

```python
# Simulation:
from applied_planning.sim.adapters import MujocoLite6Adapter
robot = MujocoLite6Adapter('model.xml', viewer=False, render_mode='offscreen')

# Real Robot:
from applied_planning.hardware import XArmLite6Adapter
robot = XArmLite6Adapter(robot_ip='192.168.1.161')

# Everything else is IDENTICAL!
robot.reset()
robot.step({'qvel': [0.1, 0, 0, 0, 0, 0]})
robot.get_ee_position()
robot.close()
```